Imports

In [14]:
import warnings
warnings.filterwarnings('ignore')

Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

Input Directory
- This is the BIDS-style directory which was created by Notebook 02

In [15]:
# Shared Base Directory
base_directory = r'/Volumes/Expansion/datasets/Jung_TMS_AD/derivatives'

This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [16]:
shared_glob_path = r'*/*/thresholded_tissue_segment_z_scores'

This is the list of tissue types to use from within each terminal folder. 
- tissue_to_import = ['cerebrospinal_fluid', 'white_matter', 'grey_matter']
- **Leave as unchanged if you are unsure**

In [17]:
tissue_to_import = ['*_csf-plus-gm.nii']

In [18]:
from calvin_utils.vbm_utils.loading import import_dataframes_by_tissue
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Volumes/Expansion/datasets/Jung_TMS_AD/derivatives/*/*/thresholded_tissue_segment_z_scores/**_csf-plus-gm.nii*
Imported data *_csf-plus-gm.nii data with 902629 voxels and 37 patients
Example filename per subject: sub-S38_csf-plus-gm.nii

--------------------------------



**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

This Should Often Be Left Default

In [19]:
preceding_id = 'sub-'
proceeding_id = '_csf'

In [20]:
from calvin_utils.vbm_utils.preprocessing import rename_dataframe_subjects
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  *_csf-plus-gm.nii
        S38  S31  S09  S36  S37  S06  S46  S48  S24  S12  ...  S43  S28  S30  \
0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
902624  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
902625  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
902626  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
902627  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
902628  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

        S

# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [21]:
folder_to_import_from = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/jung_adascog_testingdict'
shared_glob_pattern = 'optimized_convergent_map_best.nii.gz'

In [22]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder 

region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/jung_adascog_testingdict/optimized_convergent_map_best.nii.gz


,optimized_convergent_map_best.nii.gz
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
902624,0.0
902625,0.0
902626,0.0
902627,0.0


Extract Damage Scores Per Region of Interest

In [23]:
mask_path = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois/MNI152_T1_2mm_brain_mask.nii'

In [24]:
from calvin_utils.vbm_utils import postprocessing
damage_scorer = postprocessing.PostProcessing(mask_path)
damage_df_dict = damage_scorer.calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df, metrics=['cosine', 'sum', 'average', 'num_atrophic', 'total'])
damage_df_dict = damage_scorer.sort_dataframes_by_index(damage_df_dict)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  3.99it/s]


Save the Results

In [25]:
analysis = 'ConvergentMemnet_0mm_thresholded_zscore_atrophy'
session = 'pre-tms'
root_dir = '/Volumes/Expansion/datasets/Jung_TMS_AD'

In [26]:
damage_scorer.save_csv_to_metadata(damage_df_dict, root_dir=root_dir, analysis=analysis, ses=session, dry_run=False)

100%|██████████| 1/1 [00:00<00:00, 20.67it/s]

Saved to:  /Volumes/Expansion/datasets/Jung_TMS_AD/metadata/ConvergentMemnet_0mm_thresholded_zscore_atrophy_ses-pre-tms_sub-all__csf-plus-gm.csv


All done. Enjoy your analyses. 

--Calvin 